In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import sys, os
import platform
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print(platform.python_version())
print(tf.__version__)

assert platform.python_version()[0] == "3", \
    "You are using python 2. Please install python 3 and reinstall all packages."
assert tf.__version__ == "1.3.0", \
    "You are using version {} of Tensorflow. The version must be 1.3.0.".format(tf.__version__)
    
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

tf.reset_default_graph()

def negloglikelihood_cost(out, target_out):
    return -tf.reduce_sum(target_out * tf.log(out + 1e-7), axis=1)

def accuracy(out, target_out):
    fraction = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out, axis=1), tf.argmax(target_out, axis=1)), tf.float32))
    return fraction * 100

def fully_connected(inp, size, activation_fn, scope):
    with tf.variable_scope(scope):
        weights = tf.get_variable("w", [inp.get_shape()[1], size])
        bias = tf.get_variable("b", [size], initializer=tf.zeros_initializer())
    return activation_fn(tf.matmul(inp, weights) + bias)

x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
hidden1 = fully_connected(x, 200, tf.tanh, scope="hidden1")
hidden2 = fully_connected(hidden1, 200, tf.tanh, scope="hidden2")
y = fully_connected(hidden2, 10, tf.nn.softmax, scope="y")

target = tf.placeholder(tf.float32, shape=[None, 10], name="target")

cost = tf.reduce_mean(negloglikelihood_cost(y, target))
acc = accuracy(y, target)
gradients = tf.gradients(cost, tf.trainable_variables())

update_op = tf.group(*[tf.assign(var, var - 0.0001 * grad) for var, grad in zip(tf.trainable_variables(), gradients)])
init_op = tf.global_variables_initializer()

session = tf.Session()
session.run(init_op)

for i in range(10):
    batch_x, batch_y = mnist.train.next_batch(10)
    
    feed_dict = {
        x: batch_x,
        target: batch_y
    }
    result = session.run([cost, acc, gradients, update_op], feed_dict=feed_dict)
    
print("Great, your installation seems to be correct! No further action required.")